In [ ]:
## script de récupération des données TMF et GFC sur GEE

import yaml

def charger_fichier_yaml(nom_fichier):
    with open(nom_fichier, 'r') as fichier:
        contenu = fichier.read()
        donnees = yaml.safe_load(contenu)
    return donnees

# Exemple d'utilisation
nom_fichier_yaml = 'N:/Informatique/SIG/Application/Jupyterhub/projets/catalogFiles/data_config_files/TMF_annualChangeCollection_v12022_Dec_2021.yaml'
specs = charger_fichier_yaml(nom_fichier_yaml)
print(specs)

In [ ]:
%reload_ext autoreload
%autoreload 2
import ee
import os
import geemap
import sys
os.chdir('N:/Informatique/SIG/Etudes/2023/2309_QC_feux/Travail/Scripts/dynamic_land_cover/')
import gee as gee
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.plot import show
import pandas as pd
from shapely.geometry import Point
import numpy as np

chemin_output_csv='N:/Informatique/SIG/Etudes/2023/2309_PI_protocole_Surfor_CEO/OUTPUT_TMF/Foret_non_Foret_2021/'
data=pd.read_csv('N:/Informatique/SIG/Etudes/2023/2309_PI_protocole_Surfor_CEO/Scripts/Recup_TMF_GFC/point_foret_non_foret_2021_pixel.csv',delimiter=';')
file_output='TMF_foret_non_foret_2021.csv'

In [ ]:
ee.Initialize()

geometry = [Point(xy) for xy in zip(data['lon'], data['lat'])]

gdf = gpd.GeoDataFrame(data, geometry=geometry)
gdf.set_crs(epsg=4326, inplace=True)

### check gdf length, cela créer le nombre nécessaire de gdf tout en gardant la condition que la longueur du gdf ne doit pas dépasser 5000 lignes
if len(gdf) > 5000:
    num_chunks = int(np.ceil(len(gdf) / 5000))
    gdfs = []
    
    for i in range(num_chunks):
        start_idx = i * 5000
        end_idx = start_idx + 5000
        gdfs.append(gdf.iloc[start_idx:end_idx])
else:
    gdfs = [gdf]

In [ ]:
output=[]
for i, gdf in enumerate(gdfs, start=1):
    # Appliquer la commande d'extraction des données
    output_gdf = gee.extract_data(specs, gdf)
    print(output_gdf)

    output.append(output_gdf)
output_results=pd.concat(output,axis=0)
output_results.to_csv(chemin_output_csv+file_output)